In [ ]:
Pkg.update()

In [ ]:
## EXAMPLE: basic constrained problem

using JuMP
using Gurobi

m = Model(solver=GurobiSolver())

@variable(m, 0 <= x <= 2)
@variable(m, 0 <= y <= 30)

@objective(m, Max, 5x + 3y)
@constraint(m, 1x + 5y <= 3.0)

print(m)

status = solve(m)

println("Objective value: ", getobjectivevalue(m))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

In [ ]:
## EXAMPLE: matrix multiplication

A = [1 2 3; 4 5 6]
x_actual = [3; 1; 4]
x_current = [1; 4; 1]
b = [5; 9]
y_actual = A*x_actual + b

using JuMP
using Gurobi

m1 = Model(solver=GurobiSolver())

@variable(m1, e[1:3])
@variable(m1, x[1:3])
@variable(m1, y[1:2])
@constraint(m1, x .== x_current) # input
@constraint(m1, y .== y_actual) # output constraints
@constraint(m1, A*(x+e) + b .== y)

@objective(m1, Min, sum(e.^2))

print(m1)

status = solve(m1)

println("Objective value: ", getobjectivevalue(m1))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(e))

In [ ]:
### Exploring convolutions and cross-correlations
# Looks like built-in convolution operation doesn't really meet our needs

image = [
    1 2 3 4 5;
    2 3 4 5 6;
    3 4 5 6 7;
    4 5 6 7 8;
    5 6 7 8 9
]

filter = [
    -3 -1 1;
    0 0 0;
    3 1 -1;
]

filter_flipped = filter[end:-1:1, end:-1:1]

conv2(image, filter_flipped)

In [ ]:
## Example: convolution operation

include("nn_ops.jl")

using JuMP
using Gurobi

batch = 1
in_height = 3
in_width = 7
in_channels = 1
filter_height = 2
filter_width = 4
out_channels = 1

srand(1)
x_actual = rand(1:10, batch, in_height, in_width, in_channels)
x_current = rand(1:10, batch, in_height, in_width, in_channels)
filter = rand(1:10, filter_height, filter_width, in_channels, out_channels)
y_actual = conv2d(x_actual, filter)

m2 = Model(solver=GurobiSolver())

@variable(m2, e[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m2, x[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m2, y[1:batch, 1:in_height, 1:in_width, 1:out_channels])
@constraint(m2, x .== x_current) # input
@constraint(m2, y .== y_actual) # output constraints
@constraint(m2, conv2d(x+e, filter) .== y)

@objective(m2, Min, sum(e.^2))

print(m2)

status = solve(m2)

println("Objective value: ", getobjectivevalue(m2))
# TODO: Are jump solutions global? Can I save particular variables?
println("e = ", getvalue(e))

In [ ]:
## Example: rectified linearity

include("nn_ops.jl")

using JuMP
using Gurobi

batch = 1
in_height = 3
in_width = 7
in_channels = 1

srand(1)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)

m3 = Model(solver=GurobiSolver())

@variable(m3, x[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m3, e[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m3, y[1:batch, 1:in_height, 1:in_width, 1:in_channels])
reluconstraint(m3, x+e, y, 1000)

@constraint(m3, x .== x_current) # input

@constraint(m3, y .== 0)

@objective(m3, Min, sum((x+e).^2))

print(m3)

status = solve(m3)

println("Objective value: ", getobjectivevalue(m3))
println("x = ", getvalue(x))
println("e = ", getvalue(e))

In [1]:
## Example: max pooling

include("nn_ops.jl")

using JuMP
using Gurobi
using Base.Cartesian

batch = 1
in_height = 5 # TODO: test for non-matching in and pool heights
in_width = 10
in_channels = 1
pool_height = 3
pool_width = 2

out_height = round(Int, in_height/pool_height, RoundUp)
out_width = round(Int, in_width/pool_width, RoundUp)

srand(1)
x_current = rand(-10:10, batch, in_height, in_width, in_channels)

m4 = Model(solver=GurobiSolver())

@variable(m4, x[1:batch, 1:in_height, 1:in_width, 1:in_channels])
@variable(m4, y[1:batch, 1:out_height, 1:out_width, 1:in_channels])
maxpoolconstraint(m4, x, y, (3, 2) , 1000)

@constraint(m4, x .== x_current) # input

@objective(m4, Min, sum(y.^2))

print(m4)

status = solve(m4)

println("Objective value: ", getobjectivevalue(m4))
println("x = ", getvalue(x))
println("y = ", getvalue(y))

Min y[1,1,1,1]² + y[1,1,2,1]² + y[1,1,3,1]² + y[1,1,4,1]² + y[1,1,5,1]² + y[1,2,1,1]² + y[1,2,2,1]² + y[1,2,3,1]² + y[1,2,4,1]² + y[1,2,5,1]²
Subject to
 y[1,1,1,1] - x[1,1,1,1] + 1000 a[1] ≤ 1000
 y[1,1,1,1] - x[1,1,1,1] ≥ 0
 y[1,1,1,1] - x[1,1,2,1] + 1000 a[6] ≤ 1000
 y[1,1,1,1] - x[1,1,2,1] ≥ 0
 y[1,1,1,1] - x[1,2,1,1] + 1000 a[2] ≤ 1000
 y[1,1,1,1] - x[1,2,1,1] ≥ 0
 y[1,1,1,1] - x[1,2,2,1] + 1000 a[7] ≤ 1000
 y[1,1,1,1] - x[1,2,2,1] ≥ 0
 y[1,1,1,1] - x[1,3,1,1] + 1000 a[3] ≤ 1000
 y[1,1,1,1] - x[1,3,1,1] ≥ 0
 y[1,1,1,1] - x[1,3,2,1] + 1000 a[8] ≤ 1000
 y[1,1,1,1] - x[1,3,2,1] ≥ 0
 a[1] + a[6] + a[2] + a[7] + a[3] + a[8] = 1
 y[1,2,1,1] - x[1,4,1,1] + 1000 a[4] ≤ 1000
 y[1,2,1,1] - x[1,4,1,1] ≥ 0
 y[1,2,1,1] - x[1,4,2,1] + 1000 a[9] ≤ 1000
 y[1,2,1,1] - x[1,4,2,1] ≥ 0
 y[1,2,1,1] - x[1,5,1,1] + 1000 a[5] ≤ 1000
 y[1,2,1,1] - x[1,5,1,1] ≥ 0
 y[1,2,1,1] - x[1,5,2,1] + 1000 a[10] ≤ 1000
 y[1,2,1,1] - x[1,5,2,1] ≥ 0
 a[4] + a[9] + a[5] + a[10] = 1
 y[1,1,2,1] - x[1,1,3,1] + 1000 a[11] ≤